In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime

ahora = datetime.now().strftime('%Y-%m-%d')

Ticket = 'QQQ'

# Descargar los datos de QQQ
data = yf.download(Ticket, start="2023-11-01", end=ahora,interval='1d')

# Obtener los rendimientos diarios
returns = data['Adj Close'].pct_change().dropna()

# Configurar las simulaciones
n_simulations = 10
n_days = len(returns)

# Realizar las simulaciones de bootstrap
bootstrap_samples = np.random.choice(returns, (n_simulations, n_days), replace=True)
bootstrap_cumulated = np.cumprod(1 + bootstrap_samples, axis=1)

# Calcular el precio simulado para cada muestra de bootstrap
simulated_prices = bootstrap_cumulated * data['Adj Close'].iloc[0]

# Crear un DataFrame para las simulaciones
simulated_df = pd.DataFrame(simulated_prices.T)
simulated_df.set_index(data.index[:-1],inplace=True)

# Graficar las simulaciones
fig = go.Figure()

# Agregar líneas para cada simulación
for i in range(n_simulations):
    fig.add_trace(go.Scatter(x=data.index, y=simulated_df[i], mode='lines', line=dict(width=2), opacity=0.3))

# Agregar línea del precio real
fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Real', line=dict(color='black', width=2)))

# Configurar el gráfico
fig.update_layout(title=f'Simulaciones de Bootstrap para {Ticket}',
                  xaxis_title='Fecha',
                  yaxis_title='Precio Ajustado',
                  template='plotly_white')

# Mostrar el gráfico
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [3]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go


# Calcular los retornos diarios
data['Return'] = data['Close'].pct_change()

# Establecer un umbral para detectar eventos (e.g., 1% de cambio)
threshold = 0.01

eventos_si_no = data['Return'].abs() > threshold

# Crear una serie de eventos basada en el umbral
events = data.index[eventos_si_no]

# Crear una figura de Plotly
fig = go.Figure()

# Agregar la línea del precio de cierre
fig.add_trace(go.Scatter(
    x=data.index, 
    y=data['Close'], 
    mode='lines', 
    name='Close Price',
    line=dict(color='blue')
))

# Agregar los eventos como puntos
fig.add_trace(go.Scatter(
    x=events, 
    y=data.loc[events, 'Close'], 
    mode='markers', 
    name='Events',
    marker=dict(color='red', size=8)
))

# Personalizar el diseño
fig.update_layout(
    title='Precio de Cierre de AAPL con Eventos Detectados',
    xaxis_title='Fecha',
    yaxis_title='Precio de Cierre (USD)',
    legend_title='Leyenda',
    template='plotly_white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey'
    )
)

# Mostrar la figura
fig.show()


In [4]:
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

def funcion_intensidad(mu, alpha, beta, eventos, tiempos):
    # Calculate the current intensity of the Hawkes process
    return mu + alpha * np.sum(eventos * np.exp(-beta * (len(eventos) + 1 - tiempos)))

def simulated_hawkes_closing_prices(eventos, lambda_0, alpha, beta, initial_price, daily_volatility, num_days=30):
    number_events = len(eventos)
    
    eventos_historicos = list(eventos)  # Convert to list for modifiability

    # Create an array of times for historical events
    historical_times = np.arange(1, number_events + 1)

    # List to store closing prices
    prices = [initial_price]

    current_intensity = lambda_0  # Current intensity
    
    simulated_events = np.zeros(1)
    
    for i in range(1, num_days + 1):
        # Update the historical times by adding the new time
        historical_times = np.append(historical_times, number_events + i)

        P = 1 - np.exp(-current_intensity)
        
        price_change = 0  # Initialize price change for the day
        
        if np.random.uniform() < P:
            eventos_historicos.append(1)
            # Calculate price change based on the number of events
            event_price_change = np.random.normal(loc=0, scale=daily_volatility)
            price_change += event_price_change
            simulated_events = np.append(simulated_events, 1)
        else:
            simulated_events = np.append(simulated_events, 0)
            eventos_historicos.append(0)

        # Update the current intensity of the Hawkes process
        current_intensity = funcion_intensidad(lambda_0, alpha, beta, eventos_historicos, historical_times)
        
        price_change += np.random.normal(loc=0, scale=daily_volatility)
        new_price = prices[-1] + price_change
        prices.append(new_price)

    # Times from 1 to num_days to match prices length
    times = np.arange(1, num_days + 1) + number_events

    return times, prices, simulated_events

# Model parameters
lambda_0 = 0.5  # Base intensity
alpha = 0.8     # Event impact
beta = 1.5      # Decay rate
initial_price = 10  # Initial price
daily_volatility = 1  # Daily volatility
num_days = 140  # Number of days

# Initialize the events list
events = eventos_si_no  # Example initialization, replace with your actual 'eventos_si_no'

# Simulation
times, prices, simulated_events = simulated_hawkes_closing_prices(events, lambda_0, alpha, beta, initial_price, daily_volatility, num_days)

# Create Plotly plot
fig = go.Figure()

# Add the closing prices line plot
fig.add_trace(go.Scatter(
    x=times,
    y=prices[1:],  # Skip the initial price to match the time series length
    mode='lines',
    name='Precio de Cierre',
    line=dict(color='blue')
))

# Identify the times of events (where the events are 1)
event_times = [i + 1 for i, e in enumerate(simulated_events) if e == 1 and i < num_days]  # Adjust index to match times

# Add the scatter plot for events
fig.add_trace(go.Scatter(
    x=np.array(event_times) + len(events),
    y=[prices[i] for i in event_times],
    mode='markers',
    name='Eventos',
    marker=dict(color='red', symbol='x')
))

# Update the layout
fig.update_layout(
    title='Simulación de Precios de Cierre usando Proceso de Hawkes',
    xaxis_title='Días',
    yaxis_title='Precio',
    legend=dict(x=0, y=1),
    template='plotly_white'
)

# Show the plot
fig.show()


In [26]:
import numpy as np
import plotly.graph_objs as go
import pandas as pd  # Importamos pandas para manejar las fechas

def funcion_intensidad(mu, alpha, beta, eventos, tiempos):
    # Calcula la intensidad actual del proceso de Hawkes
    return mu + alpha * np.sum(eventos * np.exp(-beta * (len(eventos) + 1 - tiempos)))

def simulated_hawkes_closing_prices(eventos, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days=30):
    number_events = len(eventos)
    
    eventos_historicos = list(eventos)  # Convertir a lista para modificarla

    # Crear un array de tiempos para los eventos históricos
    historical_times = np.arange(1, number_events + 1)

    # Lista para almacenar precios de cierre
    prices = [initial_price]

    current_intensity = lambda_0  # Intensidad actual
    
    simulated_events = np.zeros(1)
    
    for i in range(1, num_days + 1):
        # Actualiza los tiempos históricos añadiendo el nuevo tiempo
        historical_times = np.append(historical_times, number_events + i)

        P = 1 - np.exp(-current_intensity)
        
        price_change = 0  # Inicializar el cambio de precio para el día
        
        if np.random.uniform() < P:
            eventos_historicos.append(1)
            # Calcula el cambio de precio basado en el número de eventos
            event_price_change = np.random.normal(loc=0, scale=daily_volatility)
            price_change += event_price_change
            simulated_events = np.append(simulated_events, 1)
        else:
            simulated_events = np.append(simulated_events, 0)
            eventos_historicos.append(0)

        # Actualiza la intensidad actual del proceso de Hawkes
        current_intensity = funcion_intensidad(lambda_0, alpha, beta, eventos_historicos, historical_times)
        
        price_change += np.random.normal(loc=0, scale=daily_volatility)
        new_price = prices[-1] + price_change
        prices.append(new_price)

    # Generar las fechas desde la fecha de inicio
    dates = pd.date_range(start=start_date, periods=num_days + 1)  # +1 para incluir el día inicial

    return dates, prices, simulated_events

# Parámetros del modelo
lambda_0 = 0.5  # Intensidad base
alpha = 0.8     # Impacto del evento
beta = 1.5      # Tasa de decaimiento
initial_price = 10  # Precio inicial
daily_volatility = 1  # Volatilidad diaria
num_days = 140  # Número de días
start_date = ahora  # Fecha de inicio

# Inicializar la lista de eventos
events = eventos_si_no#[0, 1, 0, 0, 1, 0, 1, 0]  # Ejemplo de inicialización, reemplaza con tu 'eventos_si_no'

# Simulación
dates, prices, simulated_events = simulated_hawkes_closing_prices(events, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days)

# Crear gráfico con Plotly
fig = go.Figure()

# Añadir la línea de precios de cierre
fig.add_trace(go.Scatter(
    x=dates,
    y=prices[1:],  # Saltar el precio inicial para ajustar la longitud de la serie temporal
    mode='lines',
    name='Precio de Cierre',
    line=dict(color='blue')
))

# Identificar los tiempos de eventos (donde los eventos son 1)
event_dates = [dates[i-1] for i, e in enumerate(simulated_events) if e == 1 and i < num_days]

# Añadir el scatter plot para eventos
fig.add_trace(go.Scatter(
    x=event_dates,
    y=[prices[i] for i, e in enumerate(simulated_events) if e == 1 and i < num_days],
    mode='markers',
    name='Eventos',
    marker=dict(color='red', symbol='x')
))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Simulación de Precios de Cierre usando Proceso de Hawkes',
    xaxis_title='Fechas',
    yaxis_title='Precio',
    legend=dict(x=0, y=1),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


In [31]:
import numpy as np
import plotly.graph_objs as go
import pandas as pd  # Importamos pandas para manejar las fechas

def funcion_intensidad(mu, alpha, beta, eventos, tiempos):
    # Calcula la intensidad actual del proceso de Hawkes
    return mu + alpha * np.sum(eventos * np.exp(-beta * (len(eventos) + 1 - tiempos)))

def simulated_hawkes_closing_prices(eventos, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days=30):
    number_events = len(eventos)
    
    eventos_historicos = list(eventos)  # Convertir a lista para modificarla

    # Crear un array de tiempos para los eventos históricos
    historical_times = np.arange(1, number_events + 1)

    # Lista para almacenar precios de cierre
    prices = [initial_price]

    current_intensity = lambda_0  # Intensidad actual
    
    simulated_events = np.zeros(1)
    
    for i in range(1, num_days + 1):
        # Actualiza los tiempos históricos añadiendo el nuevo tiempo
        historical_times = np.append(historical_times, number_events + i)

        P = 1 - np.exp(-current_intensity)
        
        price_change = 0  # Inicializar el cambio de precio para el día
        
        if np.random.uniform() < P:
            eventos_historicos.append(1)
            # Calcula el cambio de precio basado en el número de eventos
            event_price_change = np.random.normal(loc=0, scale=daily_volatility)
            price_change += event_price_change
            simulated_events = np.append(simulated_events, 1)
        else:
            simulated_events = np.append(simulated_events, 0)
            eventos_historicos.append(0)

        # Actualiza la intensidad actual del proceso de Hawkes
        current_intensity = funcion_intensidad(lambda_0, alpha, beta, eventos_historicos, historical_times)
        
        price_change += np.random.normal(loc=0, scale=daily_volatility)
        new_price = prices[-1] + price_change
        prices.append(new_price)

    # Generar las fechas desde la fecha de inicio
    dates = pd.date_range(start=start_date, periods=num_days + 1)  # +1 para incluir el día inicial

    return dates, prices, simulated_events

# Parámetros del modelo
lambda_0 = 0.5  # Intensidad base
alpha = 0.8     # Impacto del evento
beta = 1.5      # Tasa de decaimiento
initial_price = 1500  # Precio inicial
daily_volatility = 1  # Volatilidad diaria
num_days = 140  # Número de días
start_date = '2024-08-06'  # Fecha de inicio

# Inicializar la lista de eventos
events = eventos_si_no#[0, 1, 0, 0, 1, 0, 1, 0]  # Ejemplo de inicialización, reemplaza con tu 'eventos_si_no'

# Simulación de múltiples trayectorias
num_simulations = 5  # Número de simulaciones que quieres realizar
all_simulations = []  # Almacena todas las simulaciones

for _ in range(num_simulations):
    dates, prices, simulated_events = simulated_hawkes_closing_prices(events, lambda_0, alpha, beta, initial_price, daily_volatility, start_date, num_days)
    all_simulations.append((dates, prices, simulated_events))

# Crear gráfico con Plotly
fig = go.Figure()

# Añadir cada simulación como una línea separada
for idx, (dates, prices, simulated_events) in enumerate(all_simulations):
    # Añadir la línea de precios de cierre para cada simulación
    fig.add_trace(go.Scatter(
        x=dates,
        y=prices[1:],  # Saltar el precio inicial para ajustar la longitud de la serie temporal
        mode='lines',
        name=f'Simulación {idx + 1}',
        line=dict(width=1)  # Cambiar el ancho de línea si lo deseas
    ))

    # Identificar los tiempos de eventos (donde los eventos son 1)
    event_dates = [dates[i-1] for i, e in enumerate(simulated_events) if e == 1 and i < num_days]

    # Añadir el scatter plot para eventos de cada simulación
    fig.add_trace(go.Scatter(
        x=event_dates,
        y=[prices[i] for i, e in enumerate(simulated_events) if e == 1 and i < num_days],
        mode='markers',
        #name=f'Eventos Simulación {idx + 1}',
        marker=dict(color='red', symbol='x', size=5)
    ))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Simulación de Precios de Cierre usando Proceso de Hawkes',
    xaxis_title='Fechas',
    yaxis_title='Precio',
    #legend=dict(x=0, y=1),
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()
